## Telecom Churn - ML Group Case Study
### Business Objective
In the telecom industry, customers are able to choose from multiple service providers and actively switch from one operator to another. In this highly competitive market, the telecommunications industry experiences an average of 15-25% annual churn rate. Given the fact that it costs 5-10 times more to acquire a new customer than to retain an existing one, customer retention has now become even more important than customer acquisition.

For many incumbent operators, retaining high profitable customers is the number one business goal.

To reduce customer churn, telecom companies need to predict which customers are at high risk of churn.

In this project, we will analyse customer-level data of a leading telecom firm, build predictive models to identify customers at high risk of churn and identify the main indicators of churn.

The dataset contains customer-level information for a span of four consecutive months - June, July, August and September. The months are encoded as 6, 7, 8 and 9, respectively. 

The business objective is to predict the churn in the last (i.e. the ninth) month using the data (features) from the first three months. 

**Derive new features**

This is one of the most important parts of data preparation since good features are often the differentiators between good and bad models. Use business understanding to derive features that could be important indicators of churn.

**High valued customers will be defined as below**
 - Customers with prepaid connection
 - Customers who have recharged with an amount more than or equal to X, where X is the 70th percentile of the average recharge amount in the first two months
 
**The customers will tagged with Churn(1/0) based on the fourth month as follows** :
Those who have not made any calls (either incoming or outgoing) AND have not used mobile internet even once in the churn phase. The attributes you need to use to tag churners are:

    total_ic_mou_9
    total_og_mou_9
    vol_2g_mb_9
    vol_3g_mb_9

After tagging churners, remove all the attributes corresponding to the churn phase (all attributes having ‘ _9’, etc. in their names).


## Data Understanding

In [5]:
# Importing Pandas and NumPy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from scipy.stats import ttest_ind, ttest_ind_from_stats
import scipy.stats
from scipy.stats import chi2_contingency
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

ImportError: cannot import name 'rcParams' from 'matplotlib' (unknown location)

In [ ]:
# Importing all datasets
telecom = pd.read_csv("telecom_churn_data.csv")
print(telecom.shape)

#### Check for null values

In [ ]:
(telecom.isnull().sum()*100)/len(telecom.index)

In [ ]:
# Check for null values within rows

telecom.isnull().all(axis=1).sum()

### Handling Missing values

In [ ]:
# Since the below features would be used in deriving the high value customers, we decided to impute the missing values
# with 0
cols = ['total_rech_data_6', 'total_rech_data_7','total_rech_data_8','av_rech_amt_data_6', 'av_rech_amt_data_7', 'av_rech_amt_data_8']
for i in cols:
    telecom[i].fillna(0, inplace=True)

In [ ]:
# check the number of null values of columns over 30%
df = round(100*(telecom.isnull().sum()/len(telecom.index)), 2).to_frame()
cols = df[df[0]>30].T.columns.tolist()
cols

In [ ]:
#remove the columns with null values over 30%
telecom=telecom.drop(cols, axis=1)
print(telecom.shape)
telecom.info()

In [ ]:
# Check for null values after dropping the columns having null values greater than 30%
round(100*(telecom.isnull().sum()/len(telecom.index)), 2)

In [ ]:
# Remove all columns with a single unique value 
df = telecom.nunique().to_frame()
cols = df[df[0]==1].T.columns.tolist()
cols
print('Shape of the data set before dropping the columns with a single unique value', telecom.shape)
telecom.drop(cols, axis =1, inplace = True)
print('Shape of the data set after dropping the columns with a single unique value', telecom.shape)

#### Take day of month for date columns and dropping the date columns

In [ ]:
print(telecom[['date_of_last_rech_6', 'date_of_last_rech_7', 'date_of_last_rech_8', 'date_of_last_rech_9']].info())

# Converting date values to datetype
telecom['date_of_last_rech_6'] = pd.to_datetime(telecom['date_of_last_rech_6'])
telecom['date_of_last_rech_7'] = pd.to_datetime(telecom['date_of_last_rech_7'])
telecom['date_of_last_rech_8'] = pd.to_datetime(telecom['date_of_last_rech_8'])

In [ ]:
# Deriving the day value for each of the date columns
telecom['day_of_last_rech_6'] = telecom['date_of_last_rech_6'].apply(lambda x: x.day)
telecom['day_of_last_rech_7'] = telecom['date_of_last_rech_7'].apply(lambda x: x.day)
telecom['day_of_last_rech_8'] = telecom['date_of_last_rech_8'].apply(lambda x: x.day)

In [ ]:
telecom[['day_of_last_rech_6','day_of_last_rech_7','day_of_last_rech_8']].head()


In [ ]:
# Dropping the original date values
telecom.drop(['date_of_last_rech_6', 'date_of_last_rech_7', 'date_of_last_rech_8', 'date_of_last_rech_9'], axis =1, inplace = True)
print(telecom.shape)

#### Handling Missing values

In [ ]:
# Identifying all the features which have greater than or equal to 1 missing value and creating a list for those cols
df = round(100*(telecom.isnull().sum()/len(telecom.index)), 2).to_frame()
cols = df[df[0]>0].T.columns.tolist()
cols

In [ ]:
# Imputing the missing values for continuous features with mean and integer values with median
for i in cols:
    if 'day_' not in i:
        telecom.loc[np.isnan(telecom[i]), [i]] = telecom[i].mean()
    else:
        telecom.loc[np.isnan(telecom[i]), [i]] = telecom[i].median()
    

In [ ]:
# Checking for th existence of missing avlues after performing imputation
round(100*(telecom.isnull().sum()/len(telecom.index)), 2)

In [ ]:
telecom.shape

### Deriving High value customers

In [ ]:
# Calculating the total average recharge amount for data plus voice
telecom['avg_rech_amt_voc_data_6_7'] = ((telecom['total_rech_data_6']*telecom['av_rech_amt_data_6'])+
                                        (telecom['total_rech_data_7']*telecom['av_rech_amt_data_7'])+
                                        (telecom['total_rech_amt_6']+telecom['total_rech_amt_7']))/2

In [ ]:
telecom[['total_rech_data_6', 'total_rech_data_7', 'av_rech_amt_data_6', 'av_rech_amt_data_7','total_rech_amt_6', 
         'total_rech_amt_7', 'avg_rech_amt_voc_data_6_7']].head()

In [ ]:
# Identifying the 70th percentile to obtain the High value customers
telecom.avg_rech_amt_voc_data_6_7.quantile(0.70)

In [ ]:
# Subsetting the data set to obtain the High value customer based on total average recharge amount for data plus voice
# greater than 70th percentile which is 478.0
telecom = telecom[telecom['avg_rech_amt_voc_data_6_7']>=478.0]

In [ ]:
print('Shape of the data set after deriving high value customers',telecom.shape)

### Deriving the Churn (target variable)

In [ ]:
#The customers will be tagged with Churn(1/0) based on the fourth month as follows** :
#Those who have not made any calls (either incoming or outgoing) 
#AND have not used mobile internet even once in the churn phase. 
telecom['churn'] = 0
telecom.loc[(telecom.total_ic_mou_9 == 0) & (telecom.total_og_mou_9 == 0) & (telecom.vol_2g_mb_9 == 0) & (telecom.vol_3g_mb_9 == 0),"churn"]=1

In [ ]:
telecom.churn.value_counts()

In [ ]:
telecom.shape

After tagging churners, remove all the attributes corresponding to the churn phase i.e. all attributes having ‘ _9’, etc. in their names

In [ ]:
cols = [c for c in telecom.columns if '_9' not in c]
cols

In [ ]:
# Subsetting the telecom dataframe to drop the cols having their suffix as _9
telecom=telecom[cols]

# Also 'sep_vbc_3g belongs to the 9th month so dropping this column explicitly
telecom.drop('sep_vbc_3g',axis = 1, inplace = True)
print(telecom.shape)
telecom.head()

#### Deriving new features

In [ ]:
telecom.head()

In [ ]:
col_list = telecom.filter(regex='_6|_7').columns.str[:-2]
col_list.unique()

print (telecom.shape)

for idx, col in enumerate(col_list.unique()):
    if 'avg_' not in col:
        print(col)
        avg_col_name = "avg_"+col+"_av67"
        col_6 = col+"_6"
        col_7 = col+"_7"
        telecom[avg_col_name] = (telecom[col_6]  + telecom[col_7])/ 2

In [ ]:
telecom.shape

In [ ]:
telecom.head()

In [ ]:
# Dropping the columns having sufix as either _6 or _7 since we have derived the average values for the features
# in months 6 and 7 
col_list = telecom.filter(regex='_6|_7').columns

In [ ]:
col_list

In [ ]:
telecom.drop(col_list, axis=1, inplace=True)
telecom.shape

#### t-test assuming unequal variances to see if two sample means are significantly different

In [ ]:
# Creating a data frame to hold the first sample 'A' where Churned =0
telecom_not_churned = telecom.loc[telecom['churn']== 0,:]

In [ ]:
# Creating a data frame to hold the second sample 'B' where Churned =1
telecom_churned = telecom.loc[telecom['churn']== 1,:]

In [ ]:
cols = telecom.columns.tolist()

In [ ]:
# Defining the function for performing the t-test assuming unequal variances to see if two sample means are 
# significantly differnet
insignificant_cols = []
def t_test(variable):
    for x in variable:
        if x != 'churn':
            two_sample_results = scipy.stats.ttest_ind(telecom_not_churned[x],telecom_churned[x], equal_var = False)
            if two_sample_results[1]>0.05:
                print(x,'and its p_value is', two_sample_results[1])
                insignificant_cols.append(x)

In [ ]:
# Fetures having their p-value greater than 0.05 which denotes these features donot have a significant difference
# in their means across two sample groups which is churned group and not churned group
t_test(cols)

In [ ]:
insignificant_cols

In [ ]:
telecom.shape

In [ ]:
# Dropping these columns since these donot have a significant difference in means across two sample groups 
# which is churned group and not churned group and therefore will not provide any predictable power to the model
for i in insignificant_cols:
    telecom.drop(i,axis=1, inplace = True)
print(telecom.shape)

In [ ]:
cols = telecom.columns

In [ ]:
telecom.columns

In [ ]:
feature = []
p_value = []
def t_test(variable):
    for x in variable:
        if x != 'churn':
            two_sample_results = scipy.stats.ttest_ind(telecom_not_churned[x],telecom_churned[x], equal_var = False)
            print(x,'and its p_value is', two_sample_results[1])
            feature.append(x)
            p_value.append(two_sample_results[1])

In [ ]:
t_test(cols)

In [ ]:
df_t_test = pd.DataFrame(list(zip(feature, p_value)),
              columns=['feature','p_value'])

In [ ]:
df_t_test = df_t_test.sort_values(by=['p_value'])

In [ ]:
df_t_test

In [ ]:
good_phase_features=[]
for cols in telecom.columns:
    if '_av67' in cols:
        good_phase_features.append(cols)
        

In [ ]:
corr = telecom[good_phase_features].corr()
# Let's see the correlation matrix
plt.figure(figsize = (30,15))        # Size of the figure
sns.heatmap(corr,annot = True)
plt.show()

In [ ]:
# Select upper triangle of correlation matrix
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(np.bool)).T
# Find index of feature columns with correlation greater than 0.95
to_drop_good_phase = [index for index in upper.index if any(upper[index] > 0.80)]

In [ ]:
to_drop_good_phase

In [ ]:
action_phase_features=[]
for cols in telecom.columns:
    if '_8' in cols:
        action_phase_features.append(cols)

In [ ]:
corr = telecom[action_phase_features].corr()
# Let's see the correlation matrix
plt.figure(figsize = (30,15))        # Size of the figure
sns.heatmap(corr,annot = True)
plt.show()

In [ ]:
# Select upper triangle of correlation matrix
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(np.bool)).T
# Find index of feature columns with correlation greater than 0.95
to_drop_action_phase = [index for index in upper.index if any(upper[index] > 0.80)]

In [ ]:
to_drop_action_phase

In [ ]:
telecom.shape

In [ ]:
telecom.drop(to_drop_good_phase, axis =1, inplace = True)
telecom.drop(to_drop_action_phase, axis =1, inplace = True)
print(telecom.shape)

#### Correlation between Churn and other features

In [ ]:
plt.figure(figsize=(20,10))
telecom.corr()['churn'].sort_values(ascending = False).plot(kind='bar')
plt.show()

In [ ]:
telecom.info()

In [ ]:
def plot_continuous_chart(axe, title, plottype, col, df, log):
    axe.set_title(title)
    if log==True:
        axe.set_yscale('log')
    if (plottype=='d'):   
        sns.distplot(df[col],ax=axe)
    else: 
        sns.boxplot(data =df, x=col,ax=axe,orient='v')
        
def plot_univariate(vtype,col,hue =None,log=False,vertlabel=False, flipvertical=False):
    if vtype == 'continuous':
        fig, ax=plt.subplots(nrows =1,ncols=4,figsize=(15,5))
        plot_continuous_chart(ax[0], "Box Plot", 'b', col, telecom, log)
        plot_continuous_chart(ax[1], "Distribution Plot", 'd', col, telecom, log)
        plot_continuous_chart(ax[2], "Distribution Plot for churn", 'd', col, telecom[telecom.churn == 1], log)
        plot_continuous_chart(ax[3], "Distribution Plot for no churn", 'd', col, telecom[telecom.churn == 0], log)
    else:
        fig, ax = plt.subplots()
        hue_col = pd.Series(data = hue)
        width = len(telecom[col].unique()) + 4 + 2*len(hue_col.unique())
        if flipvertical==True:
            fig.set_size_inches(6 , 8)
            ax = sns.countplot(data = telecom, y= col, order=telecom[col].value_counts().index,hue = hue) 
        else:
            fig.set_size_inches(width , 12)
            ax = sns.countplot(data = telecom, x= col, order=telecom[col].value_counts().index,hue = hue) 
            for p in  ax.patches:
                if (p.get_height() > 0):
                    ax.annotate('{:1.2f}%'.format((p.get_height()*100)/float(len(telecom))), (p.get_x()+0.05, p.get_height()+20))  
    if (vertlabel== True): 
        plt.xticks(rotation=90)
    plt.show() 

In [ ]:
telecom.shape

In [ ]:
a = telecom.churn.dtype

In [ ]:
a

In [ ]:
#for cols in telecom.columns:
#    if telecom[cols].dtype != 'int64':
#        plot_univariate(vtype = 'continuous', col=cols, log=False)
#    else:
#        plot_univariate(vtype = 'categorical', col=cols, log=False)

In [ ]:
for cols in telecom.columns:
    telecom.boxplot(cols)
    plt.show()

## Train - Test Split

In [ ]:
from sklearn.model_selection import train_test_split
# Putting feature variable to X
X = telecom.drop(['churn','mobile_number'],axis=1)
# Putting response variable to y
y = telecom['churn']
y.head()

In [ ]:
# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=100)

In [ ]:
cols = X_train.columns.str[:]

In [ ]:
cols

In [ ]:
X_train.shape

In [ ]:
y_train.value_counts()

#### SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(kind = "regular")
X_tr,y_tr = sm.fit_sample(X_train,y_train)
print(X_tr.shape)
print(y_tr.shape)

In [ ]:
y_train.value_counts()

In [ ]:
type(y_train)

In [ ]:
pd.Series(y_tr).value_counts()

### Random Forest

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'max_depth': [4,8],
    'min_samples_leaf': range(100, 400, 200),
    'min_samples_split': range(200, 500, 200),
    'n_estimators': [100,200], 
    'max_features': [5, 10]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1,verbose = 1, scoring = 'roc_auc')

In [ ]:
grid_search.fit(X_tr, y_tr)

In [ ]:
# printing the optimal accuracy score and hyperparameters
print('We can get accuracy of',grid_search.best_score_,'using',grid_search.best_params_)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
m1= RandomForestClassifier(n_estimators=100,max_depth=8, min_samples_leaf=100,min_samples_split=200, max_features=10, n_jobs=-1)
m1.fit(X_tr,y_tr)
m1.score(X_test,y_test)

In [ ]:
predictions = m1.predict(X_test)

In [ ]:
y_trprob = m1.predict_proba(X_tr)[:,1]
y_trainprob = m1.predict_proba(X_train)[:,1]
y_testprob = m1.predict_proba(X_test)[:,1]

In [ ]:
y_trprob

In [ ]:
y_trainprob

In [ ]:
y_testprob

In [ ]:
y_tr_pred_final = pd.DataFrame(list(zip(y_tr.tolist(), y_trprob.tolist())),
              columns=['churn','churn_prob'])

In [ ]:
y_train_pred_final = pd.DataFrame(list(zip(y_train.tolist(), y_trainprob.tolist())),
              columns=['churn','churn_prob'])

In [ ]:
y_test_pred_final = pd.DataFrame(list(zip(y_test.tolist(), y_testprob.tolist())),
              columns=['churn','churn_prob'])

In [ ]:
y_tr_pred_final.head()

In [ ]:
y_train_pred_final.head()

In [ ]:
y_test_pred_final.head()

In [ ]:
# evaluation metrics
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
print(classification_report(y_test,predictions))

In [ ]:
print(confusion_matrix(y_test,predictions))

In [ ]:
print(confusion_matrix(y_test,predictions))

In [ ]:
def draw_roc( actual, probs ):
    fpr, tpr, thresholds = metrics.roc_curve( actual, probs,
                                              drop_intermediate = False )
    auc_score = metrics.roc_auc_score( actual, probs )
    plt.figure(figsize=(5, 5))
    plt.plot( fpr, tpr, label='ROC curve (area = %0.2f)' % auc_score )
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate or [1 - True Negative Rate]')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

    return None

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve( y_tr_pred_final.churn, y_tr_pred_final.churn_prob, drop_intermediate = False )

In [ ]:
draw_roc(y_tr_pred_final.churn, y_tr_pred_final.churn_prob)

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve( y_train_pred_final.churn, y_train_pred_final.churn_prob, drop_intermediate = False )

In [ ]:
draw_roc(y_train_pred_final.churn, y_train_pred_final.churn_prob)

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test_pred_final.churn, y_test_pred_final.churn_prob, drop_intermediate = False )

In [ ]:
draw_roc(y_test_pred_final.churn, y_test_pred_final.churn_prob)

In [ ]:
# Let's create columns with different probability cutoffs 
numbers = [float(x)/10 for x in range(10)]
for i in numbers:
    y_train_pred_final[i]= y_train_pred_final.churn_prob.map(lambda x: 1 if x > i else 0)
y_train_pred_final.head()

In [ ]:
# Now let's calculate accuracy sensitivity and specificity for various probability cutoffs.
cutoff_df = pd.DataFrame( columns = ['prob','accuracy','sensi','speci', 'preci'])
from sklearn.metrics import confusion_matrix

# TP = confusion[1,1] # true positive 
# TN = confusion[0,0] # true negatives
# FP = confusion[0,1] # false positives
# FN = confusion[1,0] # false negatives

num = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for i in num:
    cm1 = metrics.confusion_matrix(y_train_pred_final.churn, y_train_pred_final[i] )
    total1=sum(sum(cm1))
    accuracy = (cm1[0,0]+cm1[1,1])/total1
    
    speci = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    sensi = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    preci = cm1[1,1]/(cm1[1,1]+cm1[0,1])
    cutoff_df.loc[i] =[ i ,accuracy,sensi,speci, preci]
print(cutoff_df)

In [ ]:
# Let's plot accuracy sensitivity and specificity for various probabilities.
cutoff_df.plot.line(x='prob', y=['accuracy','sensi','speci'])
plt.show()

In [ ]:
feature_importances = pd.DataFrame(m1.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
feature_importances

## Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
len(cols)

In [ ]:
X_tr.shape

In [ ]:
telecom.head()

In [ ]:
telecom.columns

In [ ]:
X_train.head()

In [ ]:
X_tr=pd.DataFrame(data=X_tr,columns=cols)

In [ ]:
scaler = StandardScaler()

X_tr[cols] = scaler.fit_transform(X_tr[cols])

X_tr.head()

In [ ]:
X_tr.shape

### PCA on the data

#### Note - 
- While computng the principal components, we must not include the entire dataset. Model building is all about doing well on the data we haven't seen yet!
- So we'll calculate the PCs using the train data, and apply them later on the test data

In [ ]:
X_tr.shape
# We have 30 variables after creating our dummy variables for our categories

In [ ]:
X_tr.head()

In [ ]:
#Improting the PCA module
from sklearn.decomposition import PCA
pca = PCA(svd_solver='randomized', random_state=42)

In [ ]:
#Doing the PCA on the train data
pca.fit(X_tr)

#### Let's plot the principal components and try to make sense of them
- We'll plot original features on the first 2 principal components as axes

In [ ]:
pca.components_

In [ ]:
colnames = list(X_tr.columns)
pcs_df = pd.DataFrame({'PC1':pca.components_[0],'PC2':pca.components_[1], 'Feature':colnames})
pcs_df.head()

In [ ]:
%matplotlib inline
fig = plt.figure(figsize = (8,8))
plt.scatter(pcs_df.PC1, pcs_df.PC2)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
for i, txt in enumerate(pcs_df.Feature):
    plt.annotate(txt, (pcs_df.PC1[i],pcs_df.PC2[i]))
plt.tight_layout()
plt.show()

We see that the fist component is in the direction where the 'charges' variables are heavy
 - These 3 components also have the highest of the loadings

#### Looking at the screeplot to assess the number of needed principal components

In [ ]:
pca.explained_variance_ratio_

In [ ]:
#Making the screeplot - plotting the cumulative variance against the number of components
%matplotlib inline
fig = plt.figure(figsize = (12,8))
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.show()

In [ ]:
#Making the screeplot - plotting the cumulative variance against the number of components
%matplotlib inline
fig = plt.figure(figsize = (12,8))
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.show()

#### Looks like 16 components are enough to describe 95% of the variance in the dataset
- We'll choose 16 components for our modeling

In [ ]:
#Using incremental PCA for efficiency - saves a lot of time on larger datasets
from sklearn.decomposition import IncrementalPCA
pca_final = IncrementalPCA(n_components=50)

#### Basis transformation - getting the data onto our PCs

In [ ]:
df_train_pca = pca_final.fit_transform(X_tr)
df_train_pca.shape

#### Creating correlation matrix for the principal components - we expect little to no correlation

In [ ]:
#creating correlation matrix for the principal components
corrmat = np.corrcoef(df_train_pca.transpose())

In [ ]:
#plotting the correlation matrix
%matplotlib inline
plt.figure(figsize = (20,10))
sns.heatmap(corrmat,annot = True)
plt.show()

In [ ]:
# 1s -> 0s in diagonals
corrmat_nodiag = corrmat - np.diagflat(corrmat.diagonal())
print("max corr:",corrmat_nodiag.max(), ", min corr: ", corrmat_nodiag.min(),)
# we see that correlations are indeed very close to 0

#### Indeed - there is no correlation between any two components! Good job, PCA!
- We effectively have removed multicollinearity from our situation, and our models will be much more stable

In [ ]:
#Applying selected components to the test data - 16 components
df_test_pca = pca_final.transform(X_test)
df_test_pca.shape

#### Applying a logistic regression on our Principal Components
- We expect to get similar model performance with significantly lower features
- If we can do so, we would have done effective dimensionality reduction without losing any import information

In [ ]:
#Training the model on the train data
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

learner_pca = LogisticRegression()
model_pca = learner_pca.fit(df_train_pca,y_tr)

**Note**

Note that we are fitting the original variable y with the transformed variables (principal components). This is not a problem becuase the transformation done in PCA is *linear*, which implies that you've only changed the way the new x variables are represented, though the nature of relationship between X and Y is still linear. 

In [ ]:
#Making prediction on the test data
pred_probs_test = model_pca.predict_proba(df_test_pca)[:,1]
"{:2.2}".format(metrics.roc_auc_score(y_test, pred_probs_test))

#### Impressive! The same result, without all the hard work on feature selection!

Why not take it a step further and get a little more 'unsupervised' in our approach?
This time, we'll let PCA select the number of components basen on a variance cutoff we provide

In [ ]:
pca_again = PCA(0.90)

In [ ]:
df_train_pca2 = pca_again.fit_transform(X_tr)
df_train_pca2.shape
# we see that PCA selected 14 components

In [ ]:
#training the regression model
learner_pca2 = LogisticRegression()
model_pca2 = learner_pca2.fit(df_train_pca2,y_tr)

In [ ]:
df_test_pca2 = pca_again.transform(X_test)
df_test_pca2.shape

In [ ]:
#Making prediction on the test data
pred_probs_test2 = model_pca2.predict_proba(df_test_pca2)[:,1]
"{:2.2f}".format(metrics.roc_auc_score(y_test, pred_probs_test2))

#### So there it is - a very similar result, without all the hassles. We have not only achieved dimensionality reduction, but also saved a lot of effort on feature selection.

#### Before closing, let's also visualize the data to see if we can spot any patterns

In [ ]:
%matplotlib inline
fig = plt.figure(figsize = (8,8))
plt.scatter(df_train_pca[:,0], df_train_pca[:,1], c = y_train.map({0:'green',1:'red'}))
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.tight_layout()
plt.show()

Looks like there is a good amount of separation in 2D, but probably not enough

Let's look at it in 3D, and we expect spread to be better (dimensions of variance, remember?)

In [ ]:
%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(8,8))
ax = Axes3D(fig)
# ax = plt.axes(projection='3d')
ax.scatter(df_train_pca[:,2], df_train_pca[:,0], df_train_pca[:,1], c=y_train.map({0:'green',1:'red'}))

#### So let's try building the model with just 3 principal components!

In [ ]:
pca_last = PCA(n_components=50)
df_train_pca3 = pca_last.fit_transform(X_tr)
df_test_pca3 = pca_last.transform(X_test)
df_test_pca3.shape

In [ ]:
#training the regression model
learner_pca3 = LogisticRegression()
model_pca3 = learner_pca3.fit(df_train_pca3,y_tr)
#Making prediction on the test data
pred_probs_test3 = model_pca3.predict_proba(df_test_pca3)[:,1]
"{:2.2f}".format(metrics.roc_auc_score(y_test, pred_probs_test3))

#### 0.82! Isn't that just amazing!